# Twitter Sentiment Analysis

## Introduction
  
This tutorial explains a method to predict a sentiment of a twitt, in which it will be used RRNs as Deep Learning models. In general, the use of data in Social Networks is being exploited by the industry. For example, the analysis of sentiment of twits can be useful for a company that wants to analyse its new products. However, it is impossible that humans analyse each twit, therefore training deep learning models to predict is a way to scale the analysis.

The data used in this tutorial comes from an investigation made from Stanford’s researchers. They have collected 1.4 millions of twits and classify them as positive or negative emotions based on emoticons written in the same twit, simulating Facebook. The table below contains the emotion category of the emoticon used. The data was downloaded by using the HugginFace API.
  
The structure of the tutorial is divided in three parts, and it will be explained: first, the steps to clean twits in order to have a dataset to do the predictions; second, the description and use of GloVe as a Word Embedding model; third, the use of LSTM in predicting sentiments. The idea of using RRNs is that sequence of words have information that can be used to predict the sentiment.

In [1]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

## Data cleaning
In this section it will cover a step-by-step guide on how text can be cleaned, the objective is to transform the data into numbers so the models can be trained. It is necessary to understand the twit structure before designing a pipeline, below it is showed an example of a twit which was classified as negative:
  
> @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. 

  The steps to transform the data are the following:

### Remove users
This is a simple step; the objective is to identify words which contains an @ at their left side.